In [38]:
import sqlite3
import os
import json

# os.remove("/home/yicheng/remote-mount/experiments.db")
# con = sqlite3.connect("/home/yicheng/remote-mount/experiments.db")
con = sqlite3.connect(":memory:")

In [39]:
cur = con.cursor()

In [40]:
cur.execute("""\
    CREATE TABLE IF NOT EXISTS experiment(experiment_id INTEGER PRIMARY KEY, experiment_title TEXT NOT NULL)
""")

cur.execute("""\
    CREATE TABLE IF NOT EXISTS work_unit (
        experiment_id INTEGER, 
        work_unit_id INTEGER,
        PRIMARY KEY (experiment_id, work_unit_id)
    );
""")

cur.execute("""\
    CREATE TABLE IF NOT EXISTS job (
        experiment_id INTEGER,
        work_unit_id INTEGER,
        job_name TEXT,
        job_data TEXT,
        PRIMARY KEY (experiment_id, work_unit_id)
    )
""")

In [41]:
def insert_experiment(connection, experiment_id, experiment_title):
    cursor = connection.cursor()
    data = [
        (experiment_id, experiment_title)
    ]
    cursor.executemany("""
        INSERT INTO experiment VALUES
            (?, ?)
    """, data)
    connection.commit()

def insert_work_unit(connection, experiment_id, work_unit_id):
    cursor = connection.cursor()
    data = [
        (experiment_id, work_unit_id)
    ]
    cursor.executemany("""
        INSERT INTO work_unit VALUES
            (?, ?)
    """, data)
    connection.commit()

def insert_job(connection, experiment_id, work_unit_id, job_name, job_data):
    cursor = connection.cursor()
    data = [
        (experiment_id, work_unit_id, job_name, job_data)
    ]
    cursor.executemany("""
        INSERT INTO job VALUES
            (?, ?, ?, ?)
    """, data)
    connection.commit()

def list_experiment_ids(connection):
    cursor = connection.cursor()
    rows = cursor.execute("""
        SELECT experiment_id from experiment
    """)
    return rows.fetchall()

def list_work_units(connection, experiment_id):
    cursor = connection.cursor()
    rows = cursor.execute("""
        SELECT work_unit_id FROM work_unit WHERE experiment_id = ?
    """, (experiment_id, ))
    work_unit_ids = [row[0] for row in rows]
    return work_unit_ids

def get_experiment(connection, experiment_id):
    work_unit_ids = list_work_units(connection, experiment_id)
    print(work_unit_ids)
    results = {}
    for wid in work_unit_ids:
        results[wid] = get_work_unit(connection, experiment_id, wid)
    return results

def get_work_unit(connection, experiment_id, work_unit_id):
    cursor = connection.cursor()
    rows = cursor.execute("""
        SELECT job_name, job_data FROM job
        WHERE experiment_id = ?
        AND work_unit_id = ?
    """, (experiment_id, work_unit_id))
    jobs = []
    for job_name, job_data in rows:
        jobs.append({"name": job_name, **json.loads(job_data)})
    return jobs

xid = 2
wid = 1
insert_experiment(con, xid, "Hello")
insert_work_unit(con, xid, wid)
job_data = json.dumps({"data": "foo"})
insert_job(con, xid, wid, "job_name", job_data)
list_experiment_ids(con)
list_work_units(con, 1)
get_work_unit(con, 1, 1)
get_experiment(con, 1)


[]


{}

In [42]:
res = cur.execute("SELECT experiment_id FROM experiment")
res.fetchall()

[(2,)]

In [17]:
cur.execute("""
    INSERT INTO experiments (experiment_title) VALUES
        ('Test experiment')
""")
con.commit()

In [20]:
res = cur.execute("SELECT experiment_title, experiment_id FROM experiments")
res.fetchall()

[('Test experiment', 1)]